# Calc f1 score in ltr setup

In [93]:
import itertools

import click
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_ranking as tfr
from sklearn.metrics import f1_score

import logging
logger = logging.getLogger(__name__)


tf.enable_eager_execution()
tf.executing_eagerly()

# Store the paths to files containing training and test instances.
# As noted above, we will assume the data is in the LibSVM format
# and that the content of each file is sorted by query ID.

_TRAIN_DATA_PATH = ''
_TEST_DATA_PATH = ''

# Define a loss function. To find a complete list of available
# loss functions or to learn how to add your own custom function
# please refer to the tensorflow_ranking.losses module.
_LOSS = "pairwise_logistic_loss"
# _LOSS = "sigmoid_cross_entropy_loss"

# In the TF-Ranking framework, a training instance is represented
# by a Tensor that contains features from a list of documents
# associated with a single query. For simplicity, we fix the shape
# of these Tensors to a maximum list size and call it "list_size,"
# the maximum number of documents per query in the dataset.
# In this demo, we take the following approach:
#   * If a query has fewer documents, its Tensor will be padded
#     appropriately.
#   * If a query has more documents, we shuffle its list of
#     documents and trim the list down to the prescribed list_size.
_LIST_SIZE = 12

# The total number of features per query-document pair.
# We set this number to the number of features in the MSLR-Web30K
# dataset.

# Parameters to the scoring function.
_BATCH_SIZE = 100
_HIDDEN_LAYER_DIMS = ["20", "10"]


# _OUT_DIR = "../models/tfranking/"

def input_fn(path):
    train_dataset = tf.data.Dataset.from_generator(
        tfr.data.libsvm_generator(path, _NUM_FEATURES, _LIST_SIZE),
        output_types=(
            {str(k): tf.float32 for k in range(1, _NUM_FEATURES + 1)},
            tf.float32
        ),
        output_shapes=(
            {str(k): tf.TensorShape([_LIST_SIZE, 1])
             for k in range(1, _NUM_FEATURES + 1)},
            tf.TensorShape([_LIST_SIZE])
        )
    )

    train_dataset = train_dataset.batch(_BATCH_SIZE)
    return train_dataset.make_one_shot_iterator().get_next()


def example_feature_columns():
    """Returns the example feature columns."""
    feature_names = [
        "%d" % (i + 1) for i in range(0, _NUM_FEATURES)
    ]
    return {
        name: tf.feature_column.numeric_column(
            name, shape=(1,), default_value=0.0) for name in feature_names
    }


def make_score_fn():
    """Returns a scoring function to build `EstimatorSpec`."""

    def _score_fn(context_features, group_features, mode, params, config):
        """Defines the network to score a documents."""
        del params
        del config
        # Define input layer.
        example_input = [
            tf.layers.flatten(group_features[name])
            for name in sorted(example_feature_columns())
        ]
        input_layer = tf.concat(example_input, 1)

        cur_layer = input_layer
        for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
            cur_layer = tf.layers.dense(
                cur_layer,
                units=layer_width,
                activation="tanh")

        logits = tf.layers.dense(cur_layer, units=1)
        return logits

    return _score_fn


def eval_metric_fns():
    """Returns a dict from name to metric functions.

    This can be customized as follows. Care must be taken when handling padded
    lists.

    def _auc(labels, predictions, features):
    is_label_valid = tf_reshape(tf.greater_equal(labels, 0.), [-1, 1])
    clean_labels = tf.boolean_mask(tf.reshape(labels, [-1, 1], is_label_valid)
    clean_pred = tf.boolean_maks(tf.reshape(predictions, [-1, 1], is_label_valid)
    return tf.metrics.auc(clean_labels, tf.sigmoid(clean_pred), ...)
    metric_fns["auc"] = _auc

    Returns:
    A dict mapping from metric name to a metric function with above signature.
    """
    metric_fns = {}
    metric_fns.update({
        "metric/ndcg@%d" % topn: tfr.metrics.make_ranking_metric_fn(
            tfr.metrics.RankingMetricKey.NDCG, topn=topn)
        for topn in [1, 3, 5, 10]
    })

    return metric_fns


def get_estimator(hparams):
    """Create a ranking estimator.

    Args:
    hparams: (tf.contrib.training.HParams) a hyperparameters object.

    Returns:
    tf.learn `Estimator`.
    """

    def _train_op_fn(loss):
        """Defines train op used in ranking head."""
        return tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.train.get_global_step(),
            learning_rate=hparams.learning_rate,
            optimizer="Adagrad")

    ranking_head = tfr.head.create_ranking_head(
        loss_fn=tfr.losses.make_loss_fn(_LOSS),
        eval_metric_fns=eval_metric_fns(),
        train_op_fn=_train_op_fn)

    return tf.estimator.Estimator(
        model_fn=tfr.model.make_groupwise_ranking_fn(
            group_score_fn=make_score_fn(),
            group_size=1,
            transform_fn=None,
            ranking_head=ranking_head),
        params=hparams)

def create_f1_score(df, features, ranker, path):
    '''
    This function creates the f1 score. First we get the predictions on df and the file in path.
    Then we look for the maximum of the predictions and choose these transport_modes
    :param df: DataFrame where the file of path comes from
    :param features: .txt feature list with all features to use
    :param ranker: Trained model
    :param path: Path to libsvm file
    :return: DataFrame with right transport mode and yhat
    '''

    print('Start to create f1 score.')
    # Add sid to the feature set to group by later
    features = features + ['sid']

    # Predict on trained ranker
    preds = ranker.predict(input_fn=lambda: input_fn(path))

    # Go through preds generator object and append all predictions to one list
    print('Start predictions...')
    li_preds = []
    for i in preds:
        li_preds.append(i)

    # Create numpy array from list
    li_preds = np.array(li_preds)

    # df_all is an empty DataFrame to append all right f1 scores
    df_all = pd.DataFrame()

    # Iterate through all unique sids to get DataFrame
    # Iterate through number of unique sids to get yhats
    print('Creating df_all DataFrame to get just first ranking')
    for current_sid, nr_sid in zip(df.sid.unique(), range(0, df.sid.nunique())):
        print('SID: ', current_sid)
        curr_preds = li_preds[nr_sid]
        df_one_sid = df[df.sid == current_sid]
        df_one_sid = df_one_sid.assign(yhat=None)
        for current_row in range(0, len(df_one_sid)):
            df_one_sid.iloc[current_row, -1] = curr_preds[current_row]
        df_all = df_all.append(df)


    score = f1_score(df.groupby("sid").first()['click_mode'], df_all.transport_mode, average='weighted')

    print('F1 Score is: {}'.format(score))
    return df_all

In [95]:
df_train_train = pd.read_pickle("../data/processed/ranking/train_all_row_sample_50.pickle")
df_train_test = pd.read_pickle("../data/processed/ranking/train_all_row_sample_50.pickle")

_TRAIN_DATA_PATH="../data/processed/ranking/train_all_row_sample_50.libsvm"
_TEST_DATA_PATH="../data/processed/ranking/train_all_row_sample_50.libsvm"

with open('../data/processed/ranking/features_tfranking.txt') as f:
    features = f.read().splitlines()

_NUM_FEATURES = len(features)

hparams = tf.contrib.training.HParams(learning_rate=0.001)
ranker = get_estimator(hparams)

ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100)

df_preds = create_f1_score(df_train_test, features, ranker, _TEST_DATA_PATH)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpbxwqkfsh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faaa84bbe80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Checkpo

/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpbxwqkfsh/model.ckpt.
INFO:tensorflow:loss = 0.7905427, step = 1
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpbxwqkfsh/model.ckpt.
INFO:tensorflow:Loss for final step: 0.7905427.
Start to create f1 score.
Start predictions...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbxwqkfsh/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Creating df_all DataFrame to get just first ranking
SID:  10
SID:  21
SID:  25
SID:  34
SID:  35
SID:  36
SID:  44
SID:  68
SID:  69
SID:  79


ValueError: Found input variables with inconsistent numbers of samples: [10, 520]

In [96]:
# Add sid to the feature set to group by later
features = features + ['sid']

In [97]:
preds = ranker.predict(input_fn=lambda: input_fn(path))


# Go through preds generator object and append all predictions to one list
print('Start predictions...')
li_preds = []
for i in preds:
    li_preds.append(i)

# Create numpy array from list
li_preds = np.array(li_preds)


Start predictions...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbxwqkfsh/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [98]:
li_preds

array([[-1.0477258e-01, -1.0477258e-01, -1.0477258e-01, -1.0477258e-01,
         5.2324462e-01,  7.1415114e-01,  1.8639337e-05,  1.8639337e-05,
         1.8639337e-05,  1.8639337e-05,  1.8639337e-05,  1.8639337e-05],
       [-1.0477258e-01, -1.0477258e-01,  5.2324462e-01,  1.8639337e-05,
         1.8639337e-05,  1.8639337e-05,  1.8639337e-05,  1.8639337e-05,
         1.8639337e-05,  1.8639337e-05,  1.8639337e-05,  1.8639337e-05],
       [-9.9973068e-02, -1.0477258e-01, -3.3014762e-01, -6.5889239e-01,
         2.1583754e-01, -6.5889239e-01,  1.8639337e-05,  1.8639337e-05,
         1.8639337e-05,  1.8639337e-05,  1.8639337e-05,  1.8639337e-05],
       [-1.0477258e-01,  7.1415114e-01, -1.0477258e-01,  1.7805666e-02,
        -1.0477258e-01,  1.8639337e-05,  1.8639337e-05,  1.8639337e-05,
         1.8639337e-05,  1.8639337e-05,  1.8639337e-05,  1.8639337e-05],
       [ 2.1583754e-01, -2.4755168e-01, -2.4755168e-01, -1.0477258e-01,
         2.1583754e-01, -3.3014762e-01,  1.8639337e-05,  1.8

In [112]:
df = df_train_test.copy()
# df_all is an empty DataFrame to append all right f1 scores
df_all = pd.DataFrame()

# Iterate through all unique sids to get DataFrame
# Iterate through number of unique sids to get yhats
print('Creating df_all DataFrame to get just first ranking')
for current_sid, nr_sid in zip(df.sid.unique(), range(0, df.sid.nunique())):
    print('SID: ', current_sid)
    curr_preds = li_preds[nr_sid]
    df_one_sid = df[df.sid == current_sid]
    df_one_sid = df_one_sid.assign(yhat=None)
    for current_row in range(0, len(df_one_sid)):
        df_one_sid.iloc[current_row, -1] = curr_preds[current_row]
    df_all = df_all.append(df_one_sid)

Creating df_all DataFrame to get just first ranking
SID:  10
SID:  21
SID:  25
SID:  34
SID:  35
SID:  36
SID:  44
SID:  68
SID:  69
SID:  79


In [115]:
df_all = df_all.sort_values('yhat', ascending=False).drop_duplicates(['sid'])

In [129]:
y = df.groupby("sid").first()['click_mode'].values

In [139]:
df.groupby("sid").first()['click_mode']

sid
10    1
21    2
25    1
34    9
35    1
36    1
44    2
68    2
69    1
79    2
Name: click_mode, dtype: int64

In [141]:
df_all.sort_values('sid')[['sid', 'transport_mode']]

,sid,transport_mode
1432356,10,2
1435573,21,3
1460647,25,6
1440479,34,2
1449042,35,1
1440470,36,4
1448076,44,2
1456425,68,1
1455293,69,4
1459534,79,2


In [130]:
yhat = df_all.sort_values('sid')['transport_mode'].values

In [137]:
y

array([1, 2, 1, 9, 1, 1, 2, 2, 1, 2])

In [138]:
yhat

array([2, 3, 6, 2, 1, 4, 2, 1, 4, 2])

In [136]:
f1_score(y, yhat, average='weighted')

/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.34285714285714286

In [128]:
score = f1_score(df.groupby("sid").first()['click_mode'].values,
                 df_all.sort_values('sid')['transport_mode'].values,
                 average='weighted')


/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [16]:
def ltr_to_submission(df, features, ranker, path):
    features = features + ['sid']

    preds = ranker.predict(input_fn=lambda: input_fn(path))
    import itertools
    import numpy as np
    # Not sure how to get all preds because it runs infinit
    # So I take all till list size
    preds_slice = itertools.islice(preds, len(df))
    count = 0
    a = np.zeros((len(df), _LIST_SIZE))

    for i in preds_slice:
        a[count] = i
        count += 1

    test_X = df[features]

    test_X = test_X.assign(yhat=a[:, 0])

    df_end = pd.DataFrame(columns=['yhat'], index=df.sid.unique())

    df_end = test_X.sort_values(['sid', 'yhat'], ascending=False).groupby('sid').first()[[
        'yhat', 'transport_mode'
    ]]

    from sklearn.metrics import f1_score
    score = f1_score(df.groupby("sid").first()['click_mode'], df_end.transport_mode, average='weighted')
    print('F1 Score is: {}'.format(score))

    return df_end

In [30]:
path = _TEST_DATA_PATH

In [31]:
preds = ranker.predict(input_fn=lambda: input_fn(path))

In [32]:
preds

<generator object EstimatorV2.predict at 0x7faaa85a2318>

In [33]:
df_train_test.head(10)

,sid,click_time,click_mode,distance_plan,eta,price,transport_mode,plan_time_x,pid,req_time,...,max_temp,min_temp,weather,wind,weather_dy,weather_dyq,weather_q,weather_qdy,weather_xq,weather_xydy
1432355,10,2018-11-10 11:13:24,1,47429,4604,14100.0,4,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432357,10,2018-11-10 11:13:24,1,47429,4604,700.0,3,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432360,10,2018-11-10 11:13:24,1,48995,7396,3200.0,1,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432359,10,2018-11-10 11:13:24,1,47796,7234,3000.0,11,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432358,10,2018-11-10 11:13:24,1,49758,6878,5500.0,8,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432356,10,2018-11-10 11:13:24,1,49067,6345,3100.0,2,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1435574,21,2018-11-10 12:16:36,2,7667,1229,2100.0,4,2018-11-10 12:16:01,177401.0,2018-11-10 12:16:01,...,11,0,dy,12,1,0,0,0,0,0
1435572,21,2018-11-10 12:16:36,2,6157,1289,300.0,2,2018-11-10 12:16:01,177401.0,2018-11-10 12:16:01,...,11,0,dy,12,1,0,0,0,0,0
1435573,21,2018-11-10 12:16:36,2,7667,929,700.0,3,2018-11-10 12:16:01,177401.0,2018-11-10 12:16:01,...,11,0,dy,12,1,0,0,0,0,0
1460646,25,2018-11-10 23:51:30,1,2714,2434,700.0,5,2018-11-10 23:51:25,176887.0,2018-11-10 23:51:25,...,11,0,dy,12,1,0,0,0,0,0


In [34]:
a=[]
for i in preds:
    print(i)
    a.append(i)
    
a = np.array(a)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpzksxnjev/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[-1.5062747e+00 -1.5062747e+00 -9.7568709e-01 -1.5062747e+00
 -1.5062747e+00 -1.5062747e+00 -1.2835342e-04 -1.2835342e-04
 -1.2835342e-04 -1.2835342e-04 -1.2835342e-04 -1.2835342e-04]
[-1.5062747e+00 -1.4046730e+00 -1.5062747e+00 -1.2835342e-04
 -1.2835342e-04 -1.2835342e-04 -1.2835342e-04 -1.2835342e-04
 -1.2835342e-04 -1.2835342e-04 -1.2835342e-04 -1.2835342e-04]
[-1.4046730e+00 -1.1392306e+00 -8.3637077e-01 -1.1067839e+00
 -1.5062747e+00 -1.1392306e+00 -1.2835342e-04 -1.2835342e-04
 -1.2835342e-04 -1.2835342e-04 -1.2835342e-04 -1.2835342e-04]
[-1.5062747e+00 -1.5062747e+00 -1.5062747e+00 -6.2472087e-01
 -1.5062747e+00 -1.2835342e-04 -1.2835342e-04 -1.2835342e-04
 -1.2835342e-04 -1.2835

Preds are: 
* One prediction per row 
* One "array" entry per qid
* Rest is padded
* Not sure if min or max because the min in this case is the padded and max is wrong

In [40]:
a[0].argmin()

0

## Concat yhat to DataFrame

In [43]:
# Get all with the same qid e.g. 10
df = df_train_test[df_train_test.sid == 10]

In [46]:
df

,sid,click_time,click_mode,distance_plan,eta,price,transport_mode,plan_time_x,pid,req_time,...,max_temp,min_temp,weather,wind,weather_dy,weather_dyq,weather_q,weather_qdy,weather_xq,weather_xydy
1432355,10,2018-11-10 11:13:24,1,47429,4604,14100.0,4,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432357,10,2018-11-10 11:13:24,1,47429,4604,700.0,3,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432360,10,2018-11-10 11:13:24,1,48995,7396,3200.0,1,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432359,10,2018-11-10 11:13:24,1,47796,7234,3000.0,11,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432358,10,2018-11-10 11:13:24,1,49758,6878,5500.0,8,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432356,10,2018-11-10 11:13:24,1,49067,6345,3100.0,2,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0


In [57]:
df=df.assign(yhat = None)

In [73]:
df_all = pd.DataFrame()

# Iterate through all unique sids to get DataFrame
# Iterate through number of unique sids to get yhats
for current_sid, nr_sid in zip(df_train_test.sid.unique(), range(0,df_train_test.sid.nunique())):
    print('SID: {}'.format(current_sid))
    curr_preds = a[nr_sid]
    df = df_train_test[df_train_test.sid == current_sid]
    print('Length of df: {}'.format(len(df)))
    df = df.assign(yhat=None)
    for current_row in range(0, len(df)):
        df.iloc[current_row, -1] = curr_preds[current_row]
    df_all = df_all.append(df)

SID: 10
Length of df: 6
SID: 21
Length of df: 3
SID: 25
Length of df: 6
SID: 34
Length of df: 5
SID: 35
Length of df: 6
SID: 36
Length of df: 6
SID: 44
Length of df: 5
SID: 68
Length of df: 4
SID: 69
Length of df: 4
SID: 79
Length of df: 7


In [91]:
# Sort by yhat and drop duplicates -> All right preds
df_all.sort_values('yhat', ascending=False).drop_duplicates(['sid'])


,sid,click_time,click_mode,distance_plan,eta,price,transport_mode,plan_time_x,pid,req_time,...,min_temp,weather,wind,weather_dy,weather_dyq,weather_q,weather_qdy,weather_xq,weather_xydy,yhat
1440474,36,2018-11-10 13:43:03,1,3453,1801,200.0,1,2018-11-10 13:42:49,106441.0,2018-11-10 13:42:49,...,0,dy,12,1,0,0,0,0,0,-0.624721
1440480,34,2018-11-10 13:43:19,9,24642,3530,9200.0,4,2018-11-10 13:42:54,-1.0,2018-11-10 13:42:54,...,0,dy,12,1,0,0,0,0,0,-0.624721
1459537,79,2018-11-10 22:09:48,2,3479,3210,700.0,5,2018-11-10 22:09:43,148770.0,2018-11-10 22:09:43,...,0,dy,12,1,0,0,0,0,0,-0.624721
1460648,25,2018-11-10 23:51:30,1,3006,2419,200.0,1,2018-11-10 23:51:25,176887.0,2018-11-10 23:51:25,...,0,dy,12,1,0,0,0,0,0,-0.836371
1449042,35,2018-11-10 16:37:25,1,4008,2176,200.0,1,2018-11-10 16:37:17,106441.0,2018-11-10 16:37:17,...,0,dy,12,1,0,0,0,0,0,-0.836371
1432360,10,2018-11-10 11:13:24,1,48995,7396,3200.0,1,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,0,dy,12,1,0,0,0,0,0,-0.975687
1448074,44,2018-11-10 16:17:59,2,16983,2439,4700.0,4,2018-11-10 16:17:52,140368.0,2018-11-10 16:17:52,...,0,dy,12,1,0,0,0,0,0,-1.334442
1455295,69,2018-11-10 18:55:34,1,10489,2955,200.0,1,2018-11-10 18:55:23,-1.0,2018-11-10 18:55:23,...,0,dy,12,1,0,0,0,0,0,-1.404673
1435572,21,2018-11-10 12:16:36,2,6157,1289,300.0,2,2018-11-10 12:16:01,177401.0,2018-11-10 12:16:01,...,0,dy,12,1,0,0,0,0,0,-1.404673
1456423,68,2018-11-10 19:38:08,2,9977,1014,700.0,3,2018-11-10 19:38:01,-1.0,2018-11-10 19:38:01,...,0,dy,12,1,0,0,0,0,0,-1.506275
